[View in Colaboratory](https://colab.research.google.com/github/IcyCheetah358/kalevala/blob/master/tensor_poet.ipynb)

# Tensor Poet
A tensorflow deep LSTM model for text generation

In [0]:
import numpy as np
import os
import json
import random
import tensorflow as tf
from IPython.core.display import display, HTML

In [3]:
! git clone --recursive https://github.com/domschl/tensor-poet.git

Cloning into 'tensor-poet'...
remote: Counting objects: 223, done.
remote: Total 223 (delta 0), reused 0 (delta 0), pack-reused 223
Receiving objects: 100% (223/223), 1.48 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [4]:
! curl http://soinila.fi/mydeepdata/kalevala.txt -o kalevala.txt
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  564k  100  564k    0     0   564k      0  0:00:01  0:00:01 --:--:--  416k


In [53]:
!ls

datalab      kalevala.txt  tensorlog2.tar  tensor-poet.tar
example.txt  tensorlog	   tensor-poet	   tersorlog.tar


### Content
This notebook contains six sections:
1. TextLibrary: utilities to work with text files
  * loading of a list of files
  * encoding for training
  * formatted output with quote-highlighting
2. Definition of the tensorflow model
3. Model and training parameters
4. The actual training on the data (required 1. - 3.)
  * Training can be restarted, since the model is saved periodically.
5. Generation of text from the trained model (requires 1. - 4.)
6. In dialog with with the model (requires 1. - 4.

##  1. Text library

In [0]:
# TextLibrary class: text library for training, encoding, batch generation,
# and formatted source display
class TextLibrary:
    def __init__(self, filenames, max=100000000):
        self.filenames = filenames
        self.data=''
        self.files=[]
        index = 1
        for filename in filenames:
            fd={}
            fd["name"] = os.path.splitext(os.path.basename(filename))[0]
            self.c2i = {}
            self.i2c = {}
            try:
                f = open(filename)
                dat = f.read(max)
                self.data += dat
                fd["data"] = dat
                fd["index"] = index
                index += 1
                self.files.append(fd)
                f.close()
            except OSError:
                print("  ERROR: Cannot read: ", filename)
        ind = 0
        for c in self.data: # sets are not deterministic
            if c not in self.c2i:
                self.c2i[c] = ind
                self.i2c[ind] = c
                ind += 1
        self.ptr = 0
            
    def printColoredIPython(self, textlist, pre='', post=''):
        bgcolors = ['#d4e6f1', '#d8daef', '#ebdef0', '#eadbd8', '#e2d7d5', '#edebd0',
                    '#ecf3cf', '#d4efdf', '#d0ece7', '#d6eaf8', '#d4e6f1', '#d6dbdf',
                    '#f6ddcc', '#fae5d3', '#fdebd0', '#e5e8e8', '#eaeded', '#A9CCE3']
        out = ''
        for txt, ind in textlist:
            txt = txt.replace('\n','<br>')
            if ind==0:
                out += txt
            else:
                out += "<span style=\"background-color:"+bgcolors[ind%16]+";\">" + txt +\
                       "</span>"+"<sup>[" + str(ind) + "]</sup>"
        display(HTML(pre+out+post))
        
    def sourceHighlight(self, txt, minQuoteSize=10):
        tx = txt
        out = []
        qts = []
        txsrc=[("Sources: ", 0)]
        sc=False
        noquote = ''
        while len(tx)>0:  # search all library files for quote 'txt'
            mxQ = 0
            mxI = 0
            mxN = ''
            found = False
            for f in self.files:  # find longest quote in all texts
                p = minQuoteSize
                if p<=len(tx) and tx[:p] in f["data"]:
                    p = minQuoteSize + 1
                    while p<=len(tx) and tx[:p] in f["data"]:
                        p += 1
                    if p-1>mxQ:
                        mxQ = p-1
                        mxI = f["index"]
                        mxN = f["name"]
                        found = True
            if found:  # save longest quote for colorizing
                if len(noquote)>0:
                    out.append((noquote, 0))
                    noquote = ''
                out.append((tx[:mxQ],mxI))
                tx = tx[mxQ:]
                if mxI not in qts:  # create a new reference, if first occurence
                    qts.append(mxI)
                    if sc:
                        txsrc.append((", ", 0))
                    sc = True
                    txsrc.append((mxN,mxI))
            else:
                noquote += tx[0]
                tx = tx[1:]
        if len(noquote)>0:
            out.append((noquote, 0))
            noquote = ''
        self.printColoredIPython(out)
        if len(qts)>0:  # print references, if there is at least one source
            self.printColoredIPython(txsrc, pre="<small><p style=\"text-align:right;\">",
                                     post="</p></small>")
    
    def getSlice(self, length):
        if (self.ptr + length >= len(self.data)):
            self.ptr = 0
        if self.ptr == 0:
            rst = True
        else:
            rst = False
        sl = self.data[self.ptr:self.ptr+length]
        self.ptr += length
        return sl, rst
    
    def decode(self, ar):
         return ''.join([self.i2c[ic] for ic in ar])
            
    def getRandomSlice(self, length):
        p = random.randrange(0,len(self.data)-length)
        sl = self.data[p:p+length]
        return sl
    
    def getSliceArray(self, length):
        ar = np.array([c for c in self.getSlice(length)[0]])
        return ar
        
    def getSample(self, length):
        s, rst = self.getSlice(length+1)
        X = [self.c2i[c] for c in s[:-1]]
        y = [self.c2i[c] for c in s[1:]]
        return (X, y, rst)
    
    def getRandomSample(self, length):
        s = self.getRandomSlice(length+1)
        X = [self.c2i[c] for c in s[:-1]]
        y = [self.c2i[c] for c in s[1:]]
        return (X, y)
    
    def getSampleBatch(self, batch_size, length):
        smpX = []
        smpy = []
        for i in range(batch_size):
            Xi, yi, rst = self.getSample(length)
            smpX.append(Xi)
            smpy.append(yi)
        return smpX, smpy, rst
        
    def getRandomSampleBatch(self, batch_size, length):
        smpX = []
        smpy = []
        for i in range(batch_size):
            Xi, yi = self.getRandomSample(length)
            smpX.append(Xi)
            smpy.append(yi)
        return smpX, smpy

## 2. Definition of the Tensorflow model

In [0]:
# The tensorflow model for text generation
class TensorPoetModel:
    def __init__(self, params):
        self.model_name = params["model_name"]
        self.vocab_size = params["vocab_size"]
        self.neurons = params["neurons"]
        self.layers = params["layers"]
        self.learning_rate = params["learning_rate"]
        self.steps = params["steps"]
        self.logdir = params["logdir"]
        self.checkpoint = params["checkpoint"]
        # self.clip = -1.0 * params["clip"]
        
        tf.reset_default_graph()

        # Training & Generating:
        self.X = tf.placeholder(tf.int32, shape=[None, self.steps])
        self.y = tf.placeholder(tf.int32, shape=[None, self.steps])

        onehot_X = tf.one_hot(self.X, self.vocab_size)
        onehot_y = tf.one_hot(self.y, self.vocab_size)

        # Old tf 1.0 way:
        # basic_cell = tf.contrib.rnn.BasicLSTMCell(self.neurons)
        # stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * self.layers)
        # New tf 1.1 way:
        stacked_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(self.neurons, ) for _ in range(self.layers)])

        batch_size = tf.shape(self.X)[0]
        
        self.init_state_0 = stacked_cell.zero_state(batch_size, tf.float32)
        self.init_state = self.init_state_0

        with tf.variable_scope('rnn') as scope:
            rnn_outputs, states = tf.nn.dynamic_rnn(stacked_cell, onehot_X, 
                                                    initial_state=self.init_state, 
                                                    dtype=tf.float32)
            self.init_state = states

        self.final_state = self.init_state
        stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, self.neurons])

        softmax_w = tf.Variable(tf.random_normal([self.neurons, self.vocab_size]), dtype=tf.float32, name='sm_w')
        softmax_b = tf.Variable([self.vocab_size], dtype=tf.float32, name='sm_b')
            
        logits_raw = tf.matmul(stacked_rnn_outputs, softmax_w) + softmax_b
        logits = tf.reshape(logits_raw, [-1, self.steps, self.vocab_size])

        output_softmax = tf.nn.softmax(logits)

        self.temperature = tf.placeholder(tf.float32)
        self.output_softmax_temp = tf.nn.softmax(tf.div(logits, self.temperature))

        softmax_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=onehot_y, logits=logits)

        self.cross_entropy = tf.reduce_mean(softmax_entropy)
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)

        self.training_op = optimizer.minimize(self.cross_entropy)
        
        # Clipping isn't necessary, even for really deep networks:
        # grads = optimizer.compute_gradients(self.cross_entropy)
        # minclip = -1.0 * self.clip
        # capped_grads = [(tf.clip_by_value(grad, minclip, self.clip), var) for grad, var in grads]
        # self.training_op = optimizer.apply_gradients(capped_grads)

        self.prediction = tf.cast(tf.argmax(output_softmax, -1), tf.int32)
        correct_prediction = tf.equal(self.y, self.prediction)
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        error = 1.0 - self.accuracy

        
        # Tensorboard
        tf.summary.scalar("cross-entropy", self.cross_entropy)
        tf.summary.scalar("error", error)
        self.summary_merged = tf.summary.merge_all()

        # Init
        self.init = tf.global_variables_initializer()


In [8]:
!ls tensor-poet/data


README.md  tiny-shakespeare.txt


## 3. Parameters for model and training

In [0]:
libdesc = {
    "name": "TinyShakespeare",
    "description": "Small Shakespeare 'standard' corpus",
    "lib": [
        'kalevala.txt',
    ]
}

textlib = TextLibrary(libdesc["lib"])

# Model parameter:
modelParamsShakespeare = {
    "model_name": "shakespeare",
    "logdir": "tensorlog/shakespeare",
    "checkpoint": "shakespeare.ckpt",
    "vocab_size": len(textlib.i2c),
    "neurons": 256,
    "layers": 2,
    "learning_rate": 1.e-3,
    "steps": 128,
}

# Look for optional json description of a library:
if os.path.exists('bk/lib-phil-deen.json'):
    with open('bk/lib-phil-deen.json') as data_file:    
        libdescphil = json.load(data_file)
        textlib = TextLibrary(libdescphil["lib"])
        modelParamsPhil = {
            "model_name": "phil",
            "logdir": "tensorlog/phil",
            "checkpoint": "phil.ckpt",
            "vocab_size": len(textlib.i2c),
            "neurons": 256,
            "layers": 8,
            "learning_rate": 1.e-3,
            "batch_size": 128,
        }
        model = TensorPoetModel(modelParamsPhil)
else:
    model = TensorPoetModel(modelParamsShakespeare)

In [0]:
# Training Parameter:

trainParams = {
    "max_iter": 20000, #1000000,
    "restoreCheckpoints": False,
    "generateDuringTraining": True,
    "generated_text_size": 500,
    "verbose": True,
    "statusEveryNIter": 200,
    "saveEveryNIter": 500,
    "batch_size": 48, #128,
}

## 4. The actual training

In [11]:
# Run training:
with tf.Session() as sess:
    batch_size = trainParams["batch_size"]
    epl = len(textlib.data) / (batch_size * model.steps)
    model.init.run()
    tflogdir = model.logdir
    tflogdir = os.path.realpath(tflogdir)
    if not os.path.exists(tflogdir):
        os.makedirs(tflogdir)
    print("Tensorboard: 'tensorboard --logdir {}'".format(tflogdir))
    train_writer = tf.summary.FileWriter(tflogdir, sess.graph)
    train_writer.add_graph(sess.graph)
    # vl=tf.trainable_variables()
    # print(vl)
    saver = tf.train.Saver()
    checkpoint_file = os.path.join(tflogdir, model.checkpoint)
    # FFR: tf.train.export_meta_graph(filename=None, meta_info_def=None, graph_def=None, saver_def=None, collection_list=None, as_text=False, graph=None, export_scope=None, clear_devices=False, **kwargs)
    start_iter = 0
    if trainParams["restoreCheckpoints"]:
        lastSave = tf.train.latest_checkpoint(tflogdir, 
                                              latest_filename=None)
        if lastSave is not None:
            pt = lastSave.rfind('-')
            if pt != -1:
                pt += 1
                start_iter=int(lastSave[pt:])
            print("Restoring checkpoint at {}: {}".format(start_iter, lastSave))
            saver.restore(sess, lastSave)
    
    for iteration in range(start_iter, trainParams["max_iter"]):
        # Train with batches from the text library:
        X_batch, y_batch = textlib.getRandomSampleBatch(batch_size, model.steps)
        i_state = sess.run([model.init_state_0], feed_dict={model.X: X_batch})
        i_state, _ = sess.run([model.final_state, model.training_op],
                              feed_dict={model.X: X_batch, model.y: y_batch,
                                         model.init_state: i_state})

        # Output training statistics every 100 iterations:
        if iteration % 200 == 0:
            ce, accuracy, prediction, summary = sess.run([model.cross_entropy,
                                                          model.accuracy, model.prediction,
                                                          model.summary_merged],
                                             feed_dict={model.X: X_batch, model.y: y_batch})
            train_writer.add_summary(summary, iteration)
            ep = iteration / epl
            print("Epoch: {0:.2f}, iter: {1:d}, cross-entropy: {2:.3f}, accuracy: {3:.5f}".format(ep, iteration, ce, accuracy))
            if trainParams["verbose"]:
                for ind in range(1): # model.batch_size):
                    ys = textlib.decode(y_batch[ind]).replace('\n', ' | ')
                    yps = textlib.decode(prediction[ind]).replace('\n', ' | ')
                    print("   y:", ys)
                    print("  yp:", yps)

        # Generate sample texts for different temperature every ..NIter iterations:
        if (iteration+1) % trainParams["statusEveryNIter"] == 0:
            
            # Save training data
            # print("S>")
            saver.save(sess, checkpoint_file, global_step=iteration+1)
            # print("S<")

            if trainParams["generateDuringTraining"]:
                # Generate sample
                for t in range(2, 11, 4):
                    temp = float(t) / 10.0;
                    xs = ' ' * model.steps
                    xso = ''
                    doini=True
                    for i in range(trainParams["generated_text_size"]):
                        X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
                        if doini:
                            doini=False
                            g_state = sess.run([model.init_state_0], feed_dict={model.X: X_new})
                            
                        g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp], 
                                                  feed_dict={model.X: X_new, model.init_state: g_state,
                                                             model.temperature: temp})
                        inds=list(range(model.vocab_size))
                        ind = np.random.choice(inds, p=y_pred[0, -1].ravel())
                        nc = textlib.i2c[ind]
                        xso += nc
                        xs = xs[1:]+nc

                    print("----------------- temperature =", temp, "----------------------")
                    # print(xso)
                    textlib.sourceHighlight(xso, 20)   # 20: minimum quote size detected.
                print("---------------------------------------")

Tensorboard: 'tensorboard --logdir /content/tensorlog/shakespeare'
Epoch: 0.00, iter: 0, cross-entropy: 3.651, accuracy: 0.10059
   y:  parempi kuin on kesken katkemahan." |  | Niin luonen, lopettanenki, herennenki, heittänenki.  | Kerin virteni kerälle, sykkyrälle syy
  yp: ohhaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaiiaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa   a 
----------------- temperature = 0.2 ----------------------


----------------- temperature = 0.6 ----------------------


----------------- temperature = 1.0 ----------------------


---------------------------------------
Epoch: 2.21, iter: 200, cross-entropy: 2.069, accuracy: 0.36051
   y: uon sanoiksi virkki:  | "Työ minulle määrättihin aina entistä parempi:  | saa'a suuri suomuhauki, liikkuva kala lihava,  | Tuonen must
  yp: a   aanannsi karkii   | "ioinnen nee kannänää,e,,pntenvleä ea väiknii,  | "inn  siona kuunananns, kakkki i kanlnvaeenin  | loon n saot
----------------- temperature = 0.2 ----------------------


----------------- temperature = 0.6 ----------------------


...

----------------- temperature = 1.0 ----------------------


---------------------------------------
Epoch: 218.96, iter: 19800, cross-entropy: 0.249, accuracy: 0.92936
   y: i ikkunassa keihäsvartta vuolemassa,  | emo aitan kynnyksellä kirnua kolistamassa,  | veljekset veräjän suussa laitioita laatimassa,
  yp: tsktiunanta, | ukkäs arrea vuolemassa,  | emo aitan kynnyksellä kirnua kolistamassa,  | veljekset veräjän suussa laitioita laatimassa,
----------------- temperature = 0.2 ----------------------


----------------- temperature = 0.6 ----------------------


----------------- temperature = 1.0 ----------------------


---------------------------------------


In [12]:
!ls 




datalab  kalevala.txt  tensorlog  tensor-poet


## 5. Generation of text from the trained model

In [0]:
# Generating text using the model data generated during training.
def ghostWriter(textsize, temperature=1.0):
    xso = None
    with tf.Session() as sess:
        model.init.run()

        tflogdir = os.path.realpath(model.logdir)
        if not os.path.exists(tflogdir):
            print("You haven't trained a model, no data found at: {}".format(tflogdir))
            return None

        # Used for saving the training parameters periodically
        saver = tf.train.Saver()
        checkpoint_file = os.path.join(tflogdir, model.checkpoint)

        lastSave = tf.train.latest_checkpoint(tflogdir, latest_filename=None)
        if lastSave is not None:
            pt = lastSave.rfind('-')
            if pt != -1:
                pt += 1
                start_iter=int(lastSave[pt:])
            print("Restoring checkpoint at {}: {}".format(start_iter, lastSave))
            saver.restore(sess, lastSave)
        else:
            print("No checkpoints have been saved at:{}".format(trainParams["logdir"]))
            return None

        xs = ' ' * model.steps
        xso = ''
        doini = True
        for i in range(textsize):
            X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
            if doini:
                doini=False
                g_state = sess.run([model.init_state_0], feed_dict={model.X: X_new})
            g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp], 
                                      feed_dict={model.X: X_new, model.init_state: g_state,
                                                 model.temperature: temperature})
            inds=list(range(model.vocab_size))
            ind = np.random.choice(inds, p=y_pred[0, -1].ravel())
            nc = textlib.i2c[ind]
            xso += nc
            xs = xs[1:]+nc
    return(xso)


def detectPlagiarism(generatedtext, textlibrary, minQuoteLength=10):
    textlibrary.sourceHighlight(generatedtext, minQuoteLength)
    

In [52]:
tgen=ghostWriter(20000)
detectPlagiarism(tgen, textlib)

Restoring checkpoint at 20000: /content/tensorlog/shakespeare/shakespeare.ckpt-20000
INFO:tensorflow:Restoring parameters from /content/tensorlog/shakespeare/shakespeare.ckpt-20000


## 6. A dialog with the trained model

In [0]:
# Do a dialog with the recursive neural net trained above:
# def genDialogAnswer(prompt, g_state=None, endPrompt='.', maxEndPrompts=2, maxAnswerSize=512, temperature=1.0):
def doDialog():
    temperature = 0.6  # 0.1 (frozen character) - 1.3 (creative/chaotic character)
    endPrompt = '.'  # the endPrompt character is the end-mark in answers.
    maxEndPrompts = 4  # look for number of maxEndPrompts until answer is finished.
    maxAnswerSize = 2048  # Maximum length of the answer
    minAnswerSize = 64  # Minimum length of the answer

    with tf.Session() as sess:
        print("Please enter some dialog.")
        print("The net will answer according to your input.")
        print("'bye' for end,")
        print("'reset' to reset the conversation context,")
        print("'temperature=<float>' [0.1(frozen)-1.0(creative)]")
        print("    to change character of the dialog.")
        print("    Current temperature={}.".format(temperature))
        print()
        xso = None
        bye = False
        model.init.run()

        tflogdir = os.path.realpath(model.logdir)
        if not os.path.exists(tflogdir):
            print("You haven't trained a model, no data found at: {}".format(trainParams["logdir"]))
            return 

        # Used for saving the training parameters periodically
        saver = tf.train.Saver()
        checkpoint_file = os.path.join(tflogdir, model.checkpoint)

        lastSave = tf.train.latest_checkpoint(tflogdir, latest_filename=None)
        if lastSave is not None:
            pt = lastSave.rfind('-')
            if pt != -1:
                pt += 1
                start_iter=int(lastSave[pt:])
            # print("Restoring checkpoint at {}: {}".format(start_iter, lastSave))
            saver.restore(sess, lastSave)
        else:
            print("No checkpoints have been saved at:{}".format(tflogdir))
            return

        # g_state = sess.run([model.init_state_0], feed_dict={model.batch_size: 1})
        doini=True
        
        bye = False
        while not bye:
            print("> ", end="")
            prompt = input()
            if prompt == 'bye':
                bye = True
                print("Good bye!")
                continue
            if prompt == 'reset':
                doini = True
                # g_state = sess.run([model.init_state_0], feed_dict={model.batch_size: 1})
                print("(conversation context marked for reset)")
                continue
            if prompt[:len("temperature=")] == "temperature=":
                t = float(prompt[len("temperature="):])
                if t>0.05 and t<1.4:
                    temperature = t
                    print("(generator temperature now {})".format(t))
                    print()
                    continue
                print("Invalid temperature-value ignored! [0.1-1.0]")
                continue
            xs = ' ' * model.steps
            xso = ''
            for rep in range(1):
                for i in range(len(prompt)):
                    xs = xs[1:]+prompt[i]
                    X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
                    if doini:
                        doini=False
                        g_state = sess.run([model.init_state_0], feed_dict={model.X: X_new})
                    g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp], 
                                              feed_dict={model.X: X_new, model.init_state: g_state,
                                                         model.temperature: temperature})
            ans=0
            numEndPrompts = 0
            while (ans<maxAnswerSize and numEndPrompts < maxEndPrompts) or ans<minAnswerSize:

                X_new = np.transpose([[textlib.c2i[sj]] for sj in xs])
                g_state, y_pred = sess.run([model.final_state, model.output_softmax_temp], 
                                          feed_dict={model.X: X_new, model.init_state: g_state,
                                                     model.temperature: temperature})
                inds=list(range(model.vocab_size))
                ind = np.random.choice(inds, p=y_pred[0, -1].ravel())
                nc = textlib.i2c[ind]
                if nc == endPrompt:
                    numEndPrompts += 1
                xso += nc
                xs = xs[1:]+nc
                ans += 1
            print(xso.replace("\\n","\n"))
            textlib.sourceHighlight(xso, 13)
    return

In [51]:
# Talk to the net!
doDialog()

Please enter some dialog.
The net will answer according to your input.
'bye' for end,
'reset' to reset the conversation context,
'temperature=<float>' [0.1(frozen)-1.0(creative)]
    to change character of the dialog.
    Current temperature=0.6.

INFO:tensorflow:Restoring parameters from /content/tensorlog/shakespeare/shakespeare.ckpt-20000
> Ei sulla Sirpa olisi laittaa vaikka kuvaa sun klassisesta sämpyläreseptistä johon tulee auringonkukansiemeniä
!

"Jos sinä o'issa olla saanehin salahan, se on sormilta sorea, 
aina liukas liikunnolta, aina kengältä kepeä, 
liikkui sillan liitoksella, keikkui keskilattialla 
yhtä, toista toimitellen kahen kattilan välillä. 
Näki maassa palkahainen, tuosi tuohon vastaeli: 
"Pahoin tein mä, paimen parka, et hyvin, emäntä parka! 
Leivoit sie kivesi keihän, ukko uunien umani. 
Sani luoto, lempo olta, kiellä maketko kasvana 
poloisilla oppehilla lahlihoa.

Virkahti virsi vitsalla, paiskasi pajuppimille, 
kolmasti kohonnan maasta, tuon on soitan totkamm

> bye
Good bye!


In [23]:
!ls /content/tensorlog/shakespeare -la



total 52256
drwxr-xr-x 2 root root     4096 Apr  8 12:35 .
drwxr-xr-x 3 root root     4096 Apr  8 06:33 ..
-rw-r--r-- 1 root root      499 Apr  8 12:35 checkpoint
-rw-r--r-- 1 root root   799545 Apr  8 12:34 events.out.tfevents.1523169181.f498de1556d1
-rw-r--r-- 1 root root 10229780 Apr  8 12:21 shakespeare.ckpt-19200.data-00000-of-00001
-rw-r--r-- 1 root root      778 Apr  8 12:21 shakespeare.ckpt-19200.index
-rw-r--r-- 1 root root   299066 Apr  8 12:21 shakespeare.ckpt-19200.meta
-rw-r--r-- 1 root root 10229780 Apr  8 12:24 shakespeare.ckpt-19400.data-00000-of-00001
-rw-r--r-- 1 root root      778 Apr  8 12:24 shakespeare.ckpt-19400.index
-rw-r--r-- 1 root root   299066 Apr  8 12:24 shakespeare.ckpt-19400.meta
-rw-r--r-- 1 root root 10229780 Apr  8 12:28 shakespeare.ckpt-19600.data-00000-of-00001
-rw-r--r-- 1 root root      778 Apr  8 12:28 shakespeare.ckpt-19600.index
-rw-r--r-- 1 root root   299066 Apr  8 12:28 shakespeare.ckpt-19600.meta
-rw-r--r-- 1 root root 10229780 Apr  8 12:3

In [0]:
! tar -cf tensorlog2.tar tensorlog




In [50]:
! ls -la datalab


total 12
drwxr-xr-x 1 root root 4096 Mar 13 21:48 .
drwxr-xr-x 1 root root 4096 Apr  8 14:01 ..
drwxr-xr-x 1 root root 4096 Apr  8 06:22 .config


In [0]:
from google.colab import files

#with open('tensorlog.tar', 'r') as f:
#  print("Hello")
  #f.write('some content')

files.download('tensorlog2.tar')